In [1]:
import tensorflow as tf
# import tensorflow_addons as tfa
import keras
import pandas as pd
import pickle
import string
import numpy as np
from gensim.models import KeyedVectors as word2vec
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
vectors_text_path = '../data/target_vecs.txt'
all_funcs_data_path = '../data/bcb_funcs_all.tsv'
pairs_id_path = '../data/bcb_pair_ids.pkl'

In [3]:
embeddings_dim = 384
max_sequence_length = 32
output_dim = 6

In [4]:
all_funcs = pd.read_csv(all_funcs_data_path, delimiter="\t",header=None)
with open(pairs_id_path, 'rb') as f:
    pair_ids = pickle.load(f).to_numpy()

In [5]:
all_funcs

,0,1
0,74,"static void copy(String src, String dest) ..."
1,661,"public void convert(File src, File dest) t..."
2,1362,public static int[] bubbleSort(int... a) {...
3,1363,public static int[] bubbleSortOtimizado(in...
4,2450,public void process(String dir) {\n ...
...,...,...
59613,23653940,"JarAgletClassLoader(URL codebase, Certific..."
59614,23655348,"public boolean fileCompare(String from, St..."
59615,23672349,public static byte[] calculateMD4(byte[] d...
59616,23672350,public static byte[] calculateMD5(byte[] d...


In [7]:
train_data, test_data = train_test_split(pair_ids, test_size=0.2, random_state=42, stratify=pair_ids[:,2]) 

In [8]:
code2vec = word2vec.load_word2vec_format(vectors_text_path, binary=False)

In [9]:
def preproces(s):
    s = s.lower()
    s = s.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation)))
    s = s.split(" ")
    final = [0 for _ in range(max_sequence_length)]
    counter = 0
    for word in s:
        word = word.strip()
        if len(word) > 0 and word in code2vec:
            final[counter] = code2vec.vocab[word].index
            counter += 1
        
        if counter >= max_sequence_length:
            break
    return final

In [10]:
functions = dict()
for index, row in all_funcs.iterrows():
    processed_function = preproces(row[1])
    functions[int(row[0])] = processed_function

In [11]:
def get_keras_dataset(data):
    x = [[],[]]
    y = []
    for id1,id2,label in data:
        try:
            x[0].append(functions[id1])
            x[1].append(functions[id2])
            y.append(label)
        except KeyError:
            continue
    x = np.array(x)
    print(x.shape)
    x = [x[0,:],x[1,:]]

    y = to_categorical(y)
    print(y.shape)
    
    return x, y

In [12]:
x,y  = get_keras_dataset(train_data)
test_x, test_y = get_keras_dataset(test_data)

(2, 78013, 32)
(78013, 6)
(2, 19502, 32)
(19502, 6)


In [13]:
from keras.layers import *
from keras.models import *

In [44]:
vocab_size = len(code2vec.index2word)

In [45]:
embedding_layer = code2vec.get_keras_embedding(train_embeddings=False)
embedding_layer = Embedding(vocab_size, output_dim=embeddings_dim, input_length=max_sequence_length)

lstm_layer = LSTM(embeddings_dim, dropout=0.3, recurrent_dropout=0.3)

in_1 = Input(shape=(max_sequence_length,), dtype='int32')
emb_1 = embedding_layer(in_1)
lstm_1 = lstm_layer(emb_1)

in_2 = Input(shape=(max_sequence_length,), dtype='int32')
emb_2 = embedding_layer(in_2)
lstm_2 = lstm_layer(emb_2)


merged = concatenate([lstm_1, lstm_2])
merged = BatchNormalization()(merged)
merged = Dropout(0.3)(merged)
merged = Dense(512, activation='relu')(merged)
merged = BatchNormalization()(merged)
merged = Dropout(0.3)(merged)
preds = Dense(output_dim, activation='softmax')(merged)

model = Model(inputs=[in_1, in_2], outputs=preds)

In [46]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam', 
              metrics=['acc',tf.keras.metrics.Recall(),tf.keras.metrics.Precision()])

In [47]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 32)           0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 32)           0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 32, 384)      117451392   input_5[0][0]                    
                                                                 input_6[0][0]                    
__________________________________________________________________________________________________
lstm_3 (LSTM)                   (None, 384)          1181184     embedding_4[0][0]          

In [50]:
model.fit(x,
          y,
          initial_epoch=0,
          epochs=25,
          validation_split=0.2,
          batch_size=64,
          shuffle=True)

Train on 62410 samples, validate on 15603 samples
Epoch 1/25
62410/62410 [==============================] - 241s 4ms/step - loss: 0.9079 - acc: 0.6720 - recall_5: 0.5240 - precision_5: 0.6660 - val_loss: 0.6728 - val_acc: 0.7788 - val_recall_5: 0.6066 - val_precision_5: 0.7273
Epoch 2/25
62410/62410 [==============================] - 240s 4ms/step - loss: 0.6476 - acc: 0.7753 - recall_5: 0.6496 - precision_5: 0.7531 - val_loss: 0.5775 - val_acc: 0.7941 - val_recall_5: 0.6780 - val_precision_5: 0.7670
Epoch 3/25
62410/62410 [==============================] - 243s 4ms/step - loss: 0.5833 - acc: 0.7962 - recall_5: 0.6973 - precision_5: 0.7766 - val_loss: 0.5546 - val_acc: 0.8093 - val_recall_5: 0.7113 - val_precision_5: 0.7837
Epoch 4/25
62410/62410 [==============================] - 232s 4ms/step - loss: 0.5373 - acc: 0.8114 - recall_5: 0.7229 - precision_5: 0.7904 - val_loss: 0.5411 - val_acc: 0.8126 - val_recall_5: 0.7320 - val_precision_5: 0.7951
Epoch 5/25
62410/62410 [==============

In [51]:
model.evaluate(test_x, test_y)

19502/19502 [==============================] - 11s 583us/step


[0.8106748863417435,
 0.8386319279670715,
 0.8454188704490662,
 0.8694714903831482]